<a href="https://colab.research.google.com/github/garrettsomers/ManningLLMBook/blob/chapter2/Chapter_2_LLM_Architectures_and_Pre_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

Install the HuggingFace Transformers and Datasets libraries to run this notebook. Apache Beam is required to download the data used in the second half of the notebook.

In [ ]:
!pip install datasets transformers[sentencepiece,torch]
!pip install apache_beam

In this tutorial, we will explore the mechanics of LLM architectures, with an emphasis on the differences between masked models and causal models. In the first section, we'll examine some existing pretrained models to understand how they produce their outputs. Once we've demonstrated how LLM's are able to do what they do, we will then run an abbreviated training loop to provide a glimpse into the training process.

## Understanding Masked LM's

The first model we will look at is BERT, which is trained with masked tokens. As an example, the text below masks the word "box" from a well-known movie quote.

In [ ]:
text = "Life is like a [MASK] of chocolates."

We'll now see how BERT is able to predict the missing word. We can use HuggingFace to load a copy of the pretrained model and tokenizer.

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained("bert-base-uncased")

Next, we'll feed our example text into the tokenizer.

In [ ]:
encoded_input = tokenizer(text, return_tensors='pt')
print('input_ids:', encoded_input['input_ids'])
print('attention_mask:', encoded_input['attention_mask'])

The `input_ids` represent the tokenized output. Each integer can be mapped back to the corresponding string as follows.

In [ ]:
print(tokenizer.decode([7967]))

The model will then receive the output of the tokenizer. We can look at the BERT model to see exactly how it was constructed and what the outputs will be like.

In [ ]:
model

The model starts with an embedding of each of the 30,522 possible tokens into 768 dimensions, which at this point is simply a representation of each token without any additional information about their relationships to one another in the text. Then the encoder attention blocks are applied, updating the embeddings such that they now encode each token's contribution to the chunk of text and interactions with other tokens. Notably, this includes the masked tokens as well. The final stage is the language model head, which takes the embeddings from the masked positions back to 30,522 dimensions. Each index of this final vector corresponds to the probability that the token in that position would be the correct choice to fill the mask.

In [ ]:
model_output = model(**encoded_input)
output = model_output["logits"]

print(output.shape)

In [ ]:
tokens = encoded_input['input_ids'][0].tolist()
masked_index = tokens.index(tokenizer.mask_token_id)
logits = output[0, masked_index, :]

print(logits.shape)

In [ ]:
probs = logits.softmax(dim=-1)
values, predictions = probs.topk(5)
sequence = tokenizer.decode(predictions)

print('Top 5 predictions:', sequence)
print(values)

Printing the top 5 predictions and their respective scores, we see that BERT accurately chooses "box" as the most likely replacement for the mask token.

We'll now repeat a similar exercise with GPT-2.

## Understanding Causal LM's

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

We can examine the model again, noting the similarities to BERT. An embedding, 12 attention blocks, and a linear transformation bringing the output back to the size of the tokenizer. The tokenizer is different from BERT so we see we have more tokens this time.

In [ ]:
model

We'll use a different text example, since this model works by producing tokens sequentially rather than filling a mask.

In [ ]:
text = "Swimming at the beach is"
model_inputs = tokenizer(text, return_tensors='pt')
model_inputs

After applying the model, the information needed to predict the next token is represented by the last token. So we can access that vector by the index -1.

In [ ]:
import torch

output = model(**model_inputs)
next_token_logits = output.logits[:, -1, :]
next_token = torch.argmax(next_token_logits, dim=-1)
print(next_token)

Now we need to add the new token to the end of the text, so we can then feed all of it back to the model and continue predicting more tokens.

In [ ]:
model_inputs['input_ids'] = torch.cat([model_inputs['input_ids'], next_token[:, None]], dim=-1)
model_inputs["attention_mask"] = torch.cat([model_inputs['attention_mask'], torch.tensor([[1]])], dim=-1)
print(model_inputs)

Here's what we have so far. The model added the word 'a' to the input text.

In [ ]:
print(tokenizer.decode(model_inputs['input_ids'][0]))

Repeating all the previous steps, we then add the word 'great'.

In [ ]:
output = model(**model_inputs)
next_token_logits = output.logits[:, -1, :]
next_token = torch.argmax(next_token_logits, dim=-1)
model_inputs['input_ids'] = torch.cat([model_inputs['input_ids'], next_token[:, None]], dim=-1)
model_inputs["attention_mask"] = torch.cat([model_inputs['attention_mask'], torch.tensor([[1]])], dim=-1)
print(tokenizer.decode(model_inputs['input_ids'][0]))

Of course, we don't actually have to write out all these steps. HuggingFace makes text generation much more convenient. We'll use the quicker approach to finish our sentence.

In [ ]:
output_generate = model.generate(**model_inputs, max_length=20, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(output_generate[0]))

## Pre-training a GPT-2 model from scratch

Next we'll train a GPT-2 model from scratch using English Wikipedia data. Note that we're only using a tiny subset of the data to demonstrate that the model is capable of learning. The exact same approach could be followed on the full dataset to train a more functional model, but that would require a lot of compute.

In [ ]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("wikipedia", "20220301.en")
ds_shuffle = dataset['train'].shuffle()

raw_datasets = DatasetDict(
    {
        "train": ds_shuffle.select(range(50)),
        "valid": ds_shuffle.select(range(50, 100))
    }
)

raw_datasets

In [ ]:
print(raw_datasets['train'][0]['text'][:200])

We'll tokenize the text, setting the context size to 128 and thus breaking each document into chunks of 128 tokens.

In [ ]:
from transformers import AutoTokenizer

context_length = 128
tokenizer = AutoTokenizer.from_pretrained("gpt2")

outputs = tokenizer(
    raw_datasets["train"][:2]["text"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

In [ ]:
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

Now we can set up the HuggingFace Trainer as follows. Since we're using such a small dataset, we'll need lots of epochs for the model to make progress because all of the parameters are randomly initialized at the outset. Typically, most LLM's are trained for only one epoch and more diverse examples.

In [ ]:
from transformers import GPT2LMHeadModel, AutoConfig

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

model = GPT2LMHeadModel(config)

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="wiki-gpt2",
    evaluation_strategy="steps",
    num_train_epochs=100
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"]
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

The training loss is low by the end, which means the model should perform very well on training examples it has seen. It does not generalize well to the validation set of course, since we deliberately overfit on a small train set.

We can confirm with a couple of examples that were seen in training.

In [ ]:
text = tokenizer.decode(tokenized_datasets["train"][0]['input_ids'][:16])
print(text)

In [ ]:
model_inputs = tokenizer(text, return_tensors='pt')
print(model_inputs['input_ids'].shape)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_inputs['input_ids'] = model_inputs['input_ids'].to(device)
model_inputs['attention_mask'] = model_inputs['attention_mask'].to(device)

output_generate = model.generate(**model_inputs, max_new_tokens=16)
output_generate

In [ ]:
sequence = tokenizer.decode(output_generate[0])
print(sequence)

The model should do quite well at reciting text after seeing it so many times. We can be convinced that the tokenizer, model architecture, and training objective are well-suited to learning Wikipedia data. For comparison, we'll try this model on text from the validation set.

In [ ]:
text = tokenizer.decode(tokenized_datasets["valid"][0]['input_ids'][:32])
print(text)

In [ ]:
model_inputs = tokenizer(text, return_tensors='pt')

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_inputs['input_ids'] = model_inputs['input_ids'].to(device)
model_inputs['attention_mask'] = model_inputs['attention_mask'].to(device)

output_generate = model.generate(**model_inputs, max_new_tokens=16)
sequence = tokenizer.decode(output_generate[0])
print(sequence)

In [ ]:
raw_datasets['valid'][0]['text']

As expected, our model is completely confused this time. We'd need to train for much longer, and on much more diverse data, before we would have a model that can sensibly complete prompts it has never seen before. This is precisely why pre-training is such an important and powerful technique. If we had to train on all of Wikipedia for every NLP application to achieve optimal performance, it would be prohibitively expensive. But there's no need to do that when we can share and reuse existing pre-trained models as we did in the first part of this tutorial.